In [2]:
pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 3.3 MB/s 
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 51 kB 4.4 MB/s 
     |████████████████████████████████| 4.9 MB 39.6 MB/s 
     |████████████████████████████████| 1.3 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 40.4 MB/s 
     |████████████████████████████████| 6.6 MB 65.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=dfcf88275e93285dd5d4005a10d4d3a632b55bb26ab4957b7637eb698de6eee4
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
import pandas as pd
import numpy as np
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from sklearn.cluster import MeanShift
from sklearn.neighbors import NearestNeighbors as NN
from tqdm import tqdm
from scipy import spatial
from sklearn.cluster import DBSCAN, AgglomerativeClustering

In [4]:
class Pipeline_clean():
    def __init__(self, df,
                 emb_model,
                 cluster_model):
        self.df = df
        self.three_gramms = []
        self.three_gramms_united = []
        self.embeddings = []
        self.emb_model = emb_model
        self.cluster_model = cluster_model
        self.cluster_centers = []
        self.cluster_labels = []
        self.min_article_indices = []
        self.cluster_counts = dict() # {'cluster1' : 5, 'cluster2' : 21, ..., 'cluster_n' : 12}
        self.num_clusters = None

    def _get_embeddings(self):
        for text in tqdm(self.df['content']):
            self.embeddings.append(self.emb_model.encode(text))

    def _calculate_clusters(self):
        clustering = self.cluster_model.fit(self.embeddings)
        self.cluster_labels.extend(clustering.labels_)
        self.num_clusters = len(set(self.cluster_labels))

    def _get_cluster_centers(self):
        # выделили n_clusters центров
        for cluster_idx in range(self.num_clusters):
            self.embeddings = np.array(self.embeddings)
            emb_center = self.embeddings[np.where(np.array(self.cluster_labels) == cluster_idx)[0]].mean(axis=0) # len == 312
            self.cluster_centers.append(emb_center)
            # loop for each cluster and find closest embedding to the cluster_center
            min_dist = 1e10
            min_article_idx = None
            for emb_idx in np.where(np.array(self.cluster_labels) == cluster_idx)[0]:
                cos_dist = spatial.distance.cosine(self.embeddings[emb_idx], emb_center)
                if cos_dist < min_dist:
                    min_dist = cos_dist
                    min_article_idx = emb_idx
            self.min_article_indices.append(min_article_idx // 5)
            
            self.cluster_counts['cluster_' + str(cluster_idx)] = len(np.where(np.array(self.cluster_labels) == cluster_idx)[0])

    # Обрабатывает эмбеддинги новых данных и распределяет их по уже найденным кластерам
    def _get_cluster_entry(self, new_df):
        new_embeddings = []
        cluster_labels = []
        for text in tqdm(new_df['content']):
            new_embeddings.append(self.emb_model.encode(text))

        for embedding in new_embeddings:
            min_dist = 1e10
            min_dist_cluster = 0
            for cluster_id in range(self.num_clusters):
                cos_dist = spatial.distance.cosine(embedding, 
                                                      self.cluster_centers[cluster_id])
                if cos_dist < min_dist:
                    min_dist = cos_dist
                    min_dist_cluster = cluster_id
            cluster_labels.append(min_dist_cluster)

        return cluster_labels

    # Возвращает отсортированный список с id наибольших кластеров 
    def _get_biggest_clusters(self):
        return [int(k[8:]) for k, v in sorted(self.cluster_counts.items(), key=lambda item: item[1], reverse=True)]

In [5]:
sentence_model = SentenceTransformer('cointegrated/rubert-tiny')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/468k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cointegrated_rubert-tiny were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
df = pd.read_csv('full_news.csv')
df.dropna(subset=['content'], axis=0, inplace=True)
df['date']= pd.to_datetime(df['date'])

In [8]:
checks_number = 5
dates = pd.date_range('2021-05-02', '2022-10-09', checks_number)
dates -= pd.offsets.MonthBegin(1)
dates

DatetimeIndex(['2021-05-01 00:00:00', '2021-09-01 06:00:00',
               '2022-01-01 12:00:00', '2022-05-01 18:00:00',
               '2022-10-01 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [9]:
actual_df = df[(df['date'] >= dates[-1] - pd.offsets.MonthBegin(1)) & (df['date'] < dates[-1])]
actual_df.sort_values('date')

,Unnamed: 0,content,date,title
22548,3037,Губернатор Калининградской области Антон Алиха...,2022-09-01,Алиханов попросил Путина поручить перевести св...
22615,3000,Исполнительный директор японской крупной униве...,2022-09-01,Вице-премьер Новак и исполнительный директор M...
22614,3096,Решение России разрешить передачу долей в нефт...,2022-09-01,Токио считает участие своих компаний в «Сахали...
22613,3062,В Великобритании число бедных граждан вырастет...,2022-09-01,Великобритании предрекли появление миллионов б...
22612,3061,Пресс-секретарь президента России Дмитрий Песк...,2022-09-01,Песков возложил ответственность на Европу в си...
...,...,...,...,...
25333,558,Международный валютный фонд (МВФ) спрогнозиров...,2022-09-30,МВФ спрогнозировал дальнейшее снижение продово...
25332,557,Официальный представитель «Газпрома» Сергей Ку...,2022-09-30,Газпром: нарушения герметичности «Северных пот...
25330,554,Правительство Саудовской Аравии планирует влож...,2022-09-30,Саудовская Аравия инвестирует в развитие кибер...
25355,689,Министр энергетики РФ Николай Шульгинов посети...,2022-09-30,Глава Минэнерго России посетил предприятие En+...


In [10]:
cluster = AgglomerativeClustering(n_clusters=None, distance_threshold=8) # При 4х дает ~500 кластеров, при 20 ~20

pp = Pipeline_clean(actual_df, emb_model=sentence_model, cluster_model=cluster)
pp._get_embeddings()
pp._calculate_clusters()
pp._get_cluster_centers()

print('3-gram by clustser == ', pp.cluster_labels) # see sow many different clusters 
print('Cluster centers == ', pp.min_article_indices) # see the articles which are cluster centers
print(pp.cluster_counts)
print({k: v for k, v in sorted(pp.cluster_counts.items(), key=lambda item: item[1], reverse=True)}) # sorted cluster_counts

100%|██████████| 2861/2861 [04:29<00:00, 10.60it/s]


3-gram by clustser ==  [24, 58, 3, 114, 91, 33, 98, 91, 12, 14, 81, 91, 108, 12, 18, 33, 33, 101, 74, 102, 73, 89, 62, 2, 41, 48, 0, 87, 38, 73, 81, 1, 73, 74, 87, 123, 39, 7, 104, 102, 16, 31, 75, 36, 3, 121, 25, 2, 20, 51, 36, 36, 57, 44, 26, 33, 107, 3, 102, 53, 33, 103, 31, 106, 38, 56, 102, 102, 66, 111, 15, 8, 27, 8, 114, 48, 57, 54, 109, 62, 9, 47, 68, 10, 120, 75, 62, 14, 41, 53, 105, 110, 70, 28, 15, 74, 96, 116, 100, 21, 78, 92, 82, 78, 116, 15, 1, 106, 106, 116, 103, 78, 78, 78, 14, 3, 76, 114, 11, 65, 115, 12, 26, 81, 3, 37, 96, 62, 30, 122, 47, 10, 31, 1, 68, 6, 17, 39, 15, 91, 72, 37, 9, 7, 54, 10, 103, 11, 85, 93, 103, 101, 14, 23, 1, 15, 54, 23, 96, 93, 23, 14, 10, 78, 47, 30, 7, 17, 121, 106, 20, 112, 74, 119, 48, 109, 93, 34, 6, 39, 15, 33, 37, 111, 8, 81, 31, 14, 96, 82, 57, 96, 31, 35, 68, 103, 100, 11, 10, 92, 58, 26, 72, 103, 54, 36, 8, 94, 33, 91, 93, 98, 11, 45, 51, 36, 78, 11, 6, 39, 18, 4, 20, 54, 8, 74, 121, 76, 6, 26, 9, 91, 5, 33, 64, 78, 6, 23, 63, 18, 4, 

In [11]:
def perform_check(start_date):
    check_df = df[(df['date'] >= start_date) & (df['date'] < start_date + pd.offsets.MonthBegin(1))]
    check_df.sort_values('date').count()
    embedding_clusters = pp._get_cluster_entry(check_df)
    return embedding_clusters, len(check_df)

In [12]:
check_stats = []
news_amounts = []
for check_date in dates[:-1]:
    stat, amount = perform_check(check_date)
    check_stats.append(stat)
    news_amounts.append(amount)

100%|██████████| 1971/1971 [02:53<00:00, 11.35it/s]


In [13]:
  # Получаем статистику изменения упоминаний для каждого кластера
all_cluster_stats = []

for cluster_id in range(pp.num_clusters):
    cluster_stat = []
    for stat_id, check_stat in enumerate(check_stats):
        cluster_stat.append(check_stat.count(cluster_id) / news_amounts[stat_id])
    cluster_stat.append(pp.cluster_counts[f'cluster_{cluster_id}'] / len(actual_df))
    
    all_cluster_stats.append(cluster_stat)

In [14]:
# biggest_clusters = pp._get_biggest_clusters()
biggest_clusters = [int(k[8:]) for k, v in sorted(pp.cluster_counts.items(), key=lambda item: item[1], reverse=True)]
biggest_clusters = biggest_clusters[:10]
biggest_clusters

[23, 39, 6, 5, 14, 19, 0, 33, 29, 51]

In [15]:
def eval_trend(cluster_id, all_cluster_stats):
    cluster_stat = all_cluster_stats[cluster_id]
    growth = (len(cluster_stat) - 1) * cluster_stat[-1]
    growth -= sum(cluster_stat[:-1])
    return growth

In [18]:
trend_threshhold = 0.05
trend_clusters = [cluster for cluster in biggest_clusters if eval_trend(cluster, all_cluster_stats) > trend_threshhold]

In [19]:
trend_articles_ids = [pp.min_article_indices[trend_cluster] for trend_cluster in trend_clusters]
trend_articles_ids

[55, 461, 521, 196]

In [20]:
df.iloc[trend_articles_ids]['title']

55        Илон Маск заявил о наличии синдрома Аспергера 
461    Новак прокомментировал слова министра энергети...
521    В Бельгии у двоих россиян изъяли €400 тысяч по...
196    Производители оценили перспективы роста цен на...
Name: title, dtype: object

In [21]:
keyword_model = KeyBERT(model='DeepPavlov/rubert-base-cased')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/584 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
trend_words = []
for trend_article_id in trend_articles_ids:
    trend_words.append(keyword_model.extract_keywords(df.iloc[trend_article_id]['content'], keyphrase_ngram_range=(1, 1), top_n=3))


In [29]:
trend_words

[[('экоактивистке', 0.3299), ('twitter', 0.3025), ('tesla', 0.2494)],
 [('евросоюзе', 0.3267), ('газопровода', 0.2721), ('обеспечению', 0.2628)],
 [('изъятые', 0.2481), ('отмывании', 0.2192), ('похищенных', 0.2149)],
 [('минпромторг', 0.2892), ('свекловичного', 0.2828), ('превышающей', 0.2707)]]

In [31]:
result_trends = []

for word_line in trend_words:
    for word in word_line:
        result_trends.append(word[0])

In [32]:
result_trends

['экоактивистке',
 'twitter',
 'tesla',
 'евросоюзе',
 'газопровода',
 'обеспечению',
 'изъятые',
 'отмывании',
 'похищенных',
 'минпромторг',
 'свекловичного',
 'превышающей']

In [33]:
pd.DataFrame(result_trends).to_csv('trends.csv')